# AbbVie


In [4]:
import sys
sys.path.insert(0, '../../python/')

import tabula
import numpy as np
import pandas as pd
import importlib

import helpers
importlib.reload(helpers)

from helpers import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("AbbVie_2017_Switzerland_EFPIA_Report_Final.pdf", pages='all', lattice=True)

## Format Table

In [6]:
#Rename Columns
df.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total', 'empty2']
df_export = df.copy()

#Remove rows which have no values
df_export = df_export.dropna(subset=['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total'], how='all')
df_export = df_export.dropna(subset=['name', 'location', 'country'], how='all')

#Reset Index
df_export = df_export.reset_index(drop=True)

#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Remove by string comparison
df_export = df_export[~df_export['name'].str.contains('Full name')]
df_export = df_export[~df_export['name'].str.contains('Aggregate amount')]
df_export = df_export[~df_export['name'].str.contains('Number of ')]
df_export = df_export[~df_export['name'].str.contains('% of the number of')]

#Remove \n
df_export['name'] = df_export['name'].str.replace('\\r', '')

#Add type
df_export = set_type_by_alphabetical_order(df_export)

#Change Dataypes and sum
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Turn Surname/Lastname
df_export["name"] = revert_name(df_export["name"])

#Drop empties
df_export = df_export.drop(['empty1', 'empty2'], axis='columns')

#basic string conversion
df_export = basic_string_conversion(df_export)

write_to_excel(df_export, 'export.xlsx')


saved


In [7]:
#df_export.to_csv("export.csv", sep=";")